In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the generator model
def build_generator(latent_dim, num_channels):
    model = models.Sequential()
    model.add(layers.Dense(128 * 16 * 16, input_dim=latent_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Reshape((16, 16, 128)))
    model.add(layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(num_channels, (3,3), activation='tanh', padding='same'))
    return model

# Define the discriminator model
def build_discriminator(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3,3), strides=(2,2), padding='same', input_shape=input_shape))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.4))
    model.add(layers.Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.4))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Build and compile the discriminator
image_shape = (64, 64, 3)  # Adjust according to your image dimensions
discriminator = build_discriminator(image_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), metrics=['accuracy'])

# Build the generator
latent_dim = 100  # Dimensionality of the latent space
generator = build_generator(latent_dim, image_shape[-1])

# The generator takes noise as input and generates images
z = layers.Input(shape=(latent_dim,))
image = generator(z)

# For the combined model, only train the generator
discriminator.trainable = False

# The discriminator takes generated images as input and determines validity
validity = discriminator(image)

# The combined model (stacked generator and discriminator)
combined = models.Model(z, validity)
combined.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5))

# Define function to generate and save images
def generate_and_save_images(generator, epoch, latent_dim, num_examples_to_generate=16):
    noise = np.random.normal(0, 1, (num_examples_to_generate, latent_dim))
    generated_images = generator.predict(noise)

    plt.figure(figsize=(10, 10))
    for i in range(num_examples_to_generate):
        plt.subplot(4, 4, i + 1)
        plt.imshow(generated_images[i] * 0.5 + 0.5)  # De-normalize the image
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

# Load your dataset and preprocessing steps
def load_data(dataset_dir, image_size=(64, 64), batch_size=32):
    datagen = ImageDataGenerator(rescale=1./255)
    data_generator = datagen.flow_from_directory(
        dataset_dir,
        target_size=image_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=True
    )
    return data_generator

# Training parameters
epochs = 500
batch_size = 32
latent_dim = 100  # Dimensionality of the latent space
dataset_dir = '/content/drive/MyDrive/dataset'  # Replace with the path to your Market-1501 dataset directory

# Train the GAN
data_generator = load_data(dataset_dir, batch_size=batch_size)
steps_per_epoch = data_generator.samples // batch_size
for epoch in range(epochs):
    d_losses = []
    g_losses = []
    for step in range(steps_per_epoch):
        images = data_generator.next()
        num_samples = images.shape[0]
        # Adversarial ground truths
        valid = np.ones((num_samples, 1))
        fake = np.zeros((num_samples, 1))

        # Train discriminator
        noise = np.random.normal(0, 1, (num_samples, latent_dim))
        gen_images = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(images, valid)
        d_loss_fake = discriminator.train_on_batch(gen_images, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        d_losses.append(d_loss)

        # Train generator
        noise = np.random.normal(0, 1, (num_samples, latent_dim))
        g_loss = combined.train_on_batch(noise, valid)
        g_losses.append(g_loss)

    # Print progress after each epoch
    print(f"Epoch {epoch + 1}/{epochs}, D Loss: {np.mean(d_losses, axis=0)[0]}, G Loss: {np.mean(g_losses)}")

    # Optionally, save and display generated images
    if epoch % 10 == 0:
        generate_and_save_images(generator, epoch, latent_dim)

# Optionally, save the trained models
generator.save("generator_model.h5")
discriminator.save("discriminator_model.h5")

Output hidden; open in https://colab.research.google.com to view.